This is to simulate eHR system by generating synthetic HL7 Message using Python.

In [6]:
import socket
import random
from datetime import datetime
from faker import Faker

In [2]:
def random_patient_name():
    first_names = ["John", "Jane", "Alice", "Bob", "Mary", "Mike"]
    last_names = ["Smith", "Johnson", "Williams", "Brown", "Jones"]
    return random.choice(last_names), random.choice(first_names)


In [3]:
random_patient_name()

('Williams', 'Jane')

In [9]:
def random_fake_patient_name():
    fake = Faker()
    first_name = "Fake " + fake.first_name()
    last_name = fake.last_name()
    return last_name, first_name

In [10]:
random_fake_patient_name()

('Bartlett', 'Fake Michael')

In [11]:
def generate_common_segments(now, patient_id):
    # last, first = random_patient_name()
    last, first = random_fake_patient_name()
    pid = f"PID|1||{patient_id}^^^Hospital^MR||{last}^{first}^A||19800101|M|||123 Main St^^City^ST^12345||555-1234|||M|NON|123456789"
    return pid

In [12]:
generate_common_segments(datetime.now, "1234567890")

'PID|1||1234567890^^^Hospital^MR||Bray^Fake Veronica^A||19800101|M|||123 Main St^^City^ST^12345||555-1234|||M|NON|123456789'

In [13]:
def generate_hl7_message():
    now = datetime.now().strftime('%Y%m%d%H%M%S')
    patient_id = random.randint(100000, 999999)
    visit_id = random.randint(1000, 9999)

    message_type = random.choice(["ADT^A01", "ADT^A03", "ORU^R01", "ORM^O01"])
    msh = f"MSH|^~\\&|TestSystem|Hospital|EMRSystem|Facility|{now}||{message_type}|{visit_id}|P|2.5"
    pid = generate_common_segments(now, patient_id)

    segments = [msh, pid]

    if message_type == "ADT^A01":
        nk1 = "NK1|1|Doe^Jane|SPOUSE|123 Main St^^City^ST^12345|555-9876"
        pv1 = f"PV1|1|I|W^101^1^GeneralHospital||||1234^Jones^Mary^A|||MED|||||||123456^Clerk^Amy|||{now[:8]}0800"
        segments.extend([nk1, pv1])

    elif message_type == "ADT^A03":
        pv1 = f"PV1|1|I|W^101^1^GeneralHospital||||5678^Doctor^Greg^A|||MED|||||||876543^DischargeClerk^John|||{now[:8]}1700"
        segments.append(pv1)

    elif message_type == "ORU^R01":
        obr = f"OBR|1||ORD4488^LAB|CBC^Complete Blood Count|||{now}|||||||||1234^LabTech^Lori"
        obx = f"OBX|1|NM|WBC^White Blood Cells||5.6|10^9/L|4.0-10.0|N|||F"
        segments.extend([obr, obx])

    elif message_type == "ORM^O01":
        orc = f"ORC|NW|ORD8877|||CM||||{now}|||1234^Dr^Smith"
        obr = f"OBR|1|ORD8877||XRAY^Chest X-Ray|||{now}|||||||||4321^RadTech^Ray"
        segments.extend([orc, obr])

    hl7_message = "\r".join(segments)
    return hl7_message, message_type

In [14]:
generate_hl7_message()

('MSH|^~\\&|TestSystem|Hospital|EMRSystem|Facility|20250522193832||ADT^A01|8663|P|2.5\rPID|1||344716^^^Hospital^MR||Harris^Fake Judy^A||19800101|M|||123 Main St^^City^ST^12345||555-1234|||M|NON|123456789\rNK1|1|Doe^Jane|SPOUSE|123 Main St^^City^ST^12345|555-9876\rPV1|1|I|W^101^1^GeneralHospital||||1234^Jones^Mary^A|||MED|||||||123456^Clerk^Amy|||202505220800',
 'ADT^A01')

In [17]:
hl7_message, message_type = generate_hl7_message()
mllp_message = b"\x0b" + hl7_message.encode() + b"\x1c\x0d"

print(f"----- HL7 Message (message of type {message_type}) -----")
print(hl7_message.replace("\r", "\n"))

----- HL7 Message (message of type ORU^R01) -----
MSH|^~\&|TestSystem|Hospital|EMRSystem|Facility|20250522194219||ORU^R01|2933|P|2.5
PID|1||905215^^^Hospital^MR||Parks^Fake Tracy^A||19800101|M|||123 Main St^^City^ST^12345||555-1234|||M|NON|123456789
OBR|1||ORD4488^LAB|CBC^Complete Blood Count|||20250522194219|||||||||1234^LabTech^Lori
OBX|1|NM|WBC^White Blood Cells||5.6|10^9/L|4.0-10.0|N|||F
